In [1]:
from __init__ import *

In [2]:
directory_path = "{}feature/".format(SETTING["directory_path"])

kraft.path.make(directory_path)

In [3]:
for url in ("https://ndownloader.figshare.com/files/22629137",):

    print(kraft.internet.download(url, directory_path, overwrite=SETTING["overwrite"]))

/media/kwat/CarrotCake/garden/data/cancer_cell_line/feature/22629137 exists.
/media/kwat/CarrotCake/garden/data/cancer_cell_line/feature/22629137


## Map cell line name

In [4]:
name_to_rename = {}

In [5]:
name_to_id = (
    pd.read_csv(
        "../input/cell-line-synon-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=1,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

id_to_rename = (
    pd.read_csv(
        "../input/cell-line-best-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

for name, id_ in name_to_id.items():

    name_to_rename[name.lower()] = id_to_rename[id_]

len(name_to_rename)

10448

In [6]:
for name, rename in (
    pd.read_csv(
        "../input/ice000.cols.cell_line_identity.2777.csv",
        usecols=(2, 3),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .items()
):

    name_lower = name.lower()

    if name_lower in name_to_rename:

        rename_now = name_to_rename[name_lower]

        if rename_now != rename:

            print("{}: {} ==> {}".format(name, rename_now, rename))

    name_to_rename[name_lower] = rename

len(name_to_rename)

ACH-001163: PEDS005TPFAD ==> CCLFPEDS0001T
ACH-001164: PEDS015T ==> CCLFPEDS0003T
ACH-001206: TCC466 ==> TTC466
ACH-002124: H2373 ==> NCIH2373
ACH-002125: H2461 ==> NCIH2461
ACH-002126: H2591 ==> NCIH2591


11489

In [7]:
feature_x_sample = pd.read_csv(
    "{}22629137".format(directory_path), index_col=1
).T

In [8]:
for rename, names in feature_x_sample.iloc[:3, :].items():

    for name in names.dropna():

        name_lower = name.lower()

        if name_lower in name_to_rename:

            rename_now = name_to_rename[name_lower]

            if rename_now != rename:

                print("{}: {} ==> {}".format(name, rename_now, rename))

        name_to_rename[name_lower] = rename

len(name_to_rename)

NCI-H1581: NCIH1581 ==> NCIH2077
NCI-H2077: NCIH2077 ==> NCIH1581
D283: D283 ==> D283MED
ACH-000095: D341MED ==> D341
D341MED_CENTRAL_NERVOUS_SYSTEM: D341MED ==> D341
D341MED: D341MED ==> D341
SR: SR ==> SR786
ACH-000561: TTOESOPHAGUS ==> TDOTT
TT_OESOPHAGUS: TTOESOPHAGUS ==> TDOTT
ACH-000815: KMH2HAEMATOPOIETICANDLYMPHOIDTISSUE ==> KMH2
KMH2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE: KMH2HAEMATOPOIETICANDLYMPHOIDTISSUE ==> KMH2
CHL-1: CHL1 ==> HMCB
ACH-001007: A673STAG2KO16_ENGINEERED ==> A673STAG2KO16
ACH-001008: A673STAG2KO45_ENGINEERED ==> A673STAG2KO45
ACH-001009: A673STAG2NT14_ENGINEERED ==> A673STAG2NT14
ACH-001010: A673STAG2NT23_ENGINEERED ==> A673STAG2NT23
HMCB: HMCB ==> CHL1
PSN1: PSN1 ==> GRM
LC-1F: LC1F ==> LC1SQSF
ACH-001131: MS1SKIN ==> MSDASH1
MS1_SKIN: MS1SKIN ==> MSDASH1
MS-1: MS1LUNG ==> MSDASH1
R256_CENTRAL_NERVOUS_SYSTEM: R256_CENTRAL_NERVOUS_SYSTEM ==> R256
TCC466_BONE: TCC466 ==> TTC466
ACH-001225: UMRC6 ==> UMRC6NEO
UMRC6_KIDNEY: UMRC6 ==> UMRC6NEO
UPCISCC090_UPPER_AERO

11824

In [9]:
for rename in tuple(name_to_rename.values()):

    name_to_rename[rename.lower()] = rename

len(name_to_rename)

12805

In [10]:
name_to_rename = pd.Series(name_to_rename).sort_values()

name_to_rename.index.name = "Name"

name_to_rename.name = "Rename"

name_to_rename.to_csv(
    "{}cell_line_name_rename.tsv".format(directory_path), sep="\t", header=True
)

name_to_rename

Name
ach-001435       1156QE8
1156qe8          1156QE8
1205lu            1205LU
ach-002633        1205LU
127399            127399
                 ...    
idcell:933    idCell:933
cddb:867      idCell:934
idcell:934    idCell:934
idcell:935    idCell:935
cddb:868      idCell:935
Name: Rename, Length: 12805, dtype: object

## Make feature_x_sample

### All

In [14]:
feature_x_sample.loc["Is Fibroblast", :] = tuple(
    int(bool(re.match(".*FIBROBLAST$", name)))
    for name in feature_x_sample.loc["CCLE_Name", :].to_numpy()
)

feature_x_sample.index.name = "Feature"

feature_x_sample.columns.name = "Cell Line"

kraft.pd.error_axes(feature_x_sample)

feature_x_sample.to_csv(
    "{}feature_x_sample.tsv".format(directory_path), sep="\t"
)

kraft.pd.count_value(feature_x_sample.T)

feature_x_sample

--------------------------------------------------------------------------------
DepMap_ID
ACH-000835    1
ACH-000804    1
ACH-000486    1
ACH-002199    1
ACH-000021    1
             ..
ACH-002102    1
ACH-002129    1
ACH-002278    1
ACH-000444    1
ACH-001355    1
Name: DepMap_ID, Length: 1804, dtype: int64
--------------------------------------------------------------------------------
CCLE_Name
GMS10_CENTRAL_NERVOUS_SYSTEM                 1
M040416_SKIN                                 1
JHOS2_OVARY                                  1
BDCM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE      1
SUDHL8_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE    1
                                            ..
GOTO_AUTONOMIC_GANGLIA                       1
CAL33_UPPER_AERODIGESTIVE_TRACT              1
C80_LARGE_INTESTINE                          1
ES1_BONE                                     1
ICC3_BILIARY_TRACT                           1
Name: CCLE_Name, Length: 1804, dtype: int64
-------------------------------------

Cell Line,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,A101D,C2BBE1,NCIH2077,...,CCLFUPGI0005T,HT144SKINFV1,HT144SKINFV3,HT144SKINFV2,RVH421SKINFV1,HAP1,WM3211,WM4235,M040416,M140325
Feature,,,,,,,,,,,,,,,,,,,,,
DepMap_ID,ACH-000001,ACH-000002,ACH-000003,ACH-000004,ACH-000005,ACH-000006,ACH-000007,ACH-000008,ACH-000009,ACH-000010,...,ACH-002446,ACH-002458,ACH-002459,ACH-002460,ACH-002461,ACH-002475,ACH-002508,ACH-002509,ACH-002510,ACH-002511
CCLE_Name,NIHOVCAR3_OVARY,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CACO2_LARGE_INTESTINE,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,LS513_LARGE_INTESTINE,A101D_SKIN,C2BBE1_LARGE_INTESTINE,NCIH2077_LUNG,...,CCLFUPGI0005T_STOMACH,HT144_SKIN_FV1_RESISTANT,HT144_SKIN_FV3_RESISTANT,HT144_SKIN_FV2_RESISTANT,RVH421_SKIN_FV1_RESISTANT,HAP1_ENGINEERED,WM3211_SKIN,WM4235_SKIN,M040416_SKIN,M140325_SKIN
Alias,OVCAR3,NaN,"CACO2, CaCo-2",NaN,NaN,NaN,NaN,NaN,NaN,NCI-H1581,...,NaN,HT144_Dab,HT144_Dab_Tram,HTT144_Dab_Rox,RVH421_Dab,NaN,NaN,NaN,NaN,NaN
COSMICID,905933,905938,NaN,907053,NaN,908148,907795,910921,910700,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sex,Female,Female,Male,Male,Male,Male,Male,Male,Male,Male,...,Unknown,Male,Male,Male,Male,Male,Female,Unknown,Male,Female
source,ATCC,ATCC,ATCC,DSMZ,ATCC,DSMZ,ATCC,ATCC,ATCC,Academic lab,...,CCLF,Academic lab,Academic lab,Academic lab,Academic lab,Horizon Discovery,Academic lab,Academic lab,Academic lab,Academic lab
Achilles_n_replicates,NaN,NaN,NaN,2,2,NaN,2,NaN,3,NaN,...,2,2,2,1,NaN,2,2,NaN,2,NaN
cell_line_NNMD,NaN,NaN,NaN,-3.0792,-2.40441,NaN,-2.58671,NaN,-4.13549,NaN,...,-5.09651,-3.34369,-3.13339,-3.6858,NaN,-3.14273,-4.30234,NaN,-4.24171,NaN
culture_type,NaN,NaN,NaN,Suspension,Mixed adherent and suspension,NaN,Adherent,NaN,Adherent,NaN,...,Adherent,Adherent,Adherent,Adherent,NaN,Adherent,Adherent,NaN,Adherent,NaN


### Continuous

In [15]:
continuous_feature_x_sample = feature_x_sample.loc[["age"]]

continuous_feature_x_sample.to_csv(
    "{}continuous_feature_x_sample.tsv".format(directory_path), sep="\t"
)

continuous_feature_x_sample

Cell Line,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,A101D,C2BBE1,NCIH2077,...,CCLFUPGI0005T,HT144SKINFV1,HT144SKINFV3,HT144SKINFV2,RVH421SKINFV1,HAP1,WM3211,WM4235,M040416,M140325
Feature,,,,,,,,,,,,,,,,,,,,,
age,60,35,NaN,30,30,64,63,56,72,NaN,...,NaN,29,29,29,NaN,40,34,NaN,54,64


### Binary

In [16]:
dfs = [feature_x_sample.loc[["Is Fibroblast"], :]]

for label in (
    "sex",
    "sample_collection_site",
    "primary_or_metastasis",
    "primary_disease",
    "Subtype",
    "lineage",
    "lineage_subtype",
    "lineage_sub_subtype",
    "lineage_molecular_subtype",
):

    df = kraft.pd.binarize(feature_x_sample.loc[label, :])

    df.index = (
        "({}) {}".format(label, label_) for label_ in df.index.to_numpy()
    )

    dfs.append(df)

binary_feature_x_sample = pd.concat(dfs)

binary_feature_x_sample.index.name = "Feature"

binary_feature_x_sample.to_csv(
    "{}binary_feature_x_sample.tsv".format(directory_path), sep="\t"
)

binary_feature_x_sample

Cell Line,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,A101D,C2BBE1,NCIH2077,...,CCLFUPGI0005T,HT144SKINFV1,HT144SKINFV3,HT144SKINFV2,RVH421SKINFV1,HAP1,WM3211,WM4235,M040416,M140325
Feature,,,,,,,,,,,,,,,,,,,,,
Is Fibroblast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(sex) Female,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
(sex) Male,0,0,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,0,0,1,0
(sex) Unknown,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
(sample_collection_site) ascites,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
(lineage_molecular_subtype) luminal_HER2_amp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(lineage_molecular_subtype) MSI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(lineage_molecular_subtype) PAX3_FKHR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
